In [1]:
%env CUDA_DEVICE_ORDER=PCI_BUS_ID
%env CUDA_VISIBLE_DEVICES=0,1

env: CUDA_DEVICE_ORDER=PCI_BUS_ID
env: CUDA_VISIBLE_DEVICES=0,1


In [2]:
import os
import sys
import functools
import time

import numpy as np
import pickle
import matplotlib
import matplotlib.pyplot as plt
import sklearn.metrics
from sklearn.decomposition import PCA, IncrementalPCA
from sklearn.cluster import spectral_clustering
from sklearn.svm import SVC

try:
  import torch
  import torch.nn as nn
  from torch.autograd import Variable
except ModuleNotFoundError:
  pass

solver_path = '/projects/academic/azhang/tianlema/deeplearning'
if not os.path.exists(solver_path):
  solver_path = 'I:/code'
#   solver_path = 'C:/Users/Tianle/Documents/cs231n/spring1617/my-scripts'
if not os.path.exists(solver_path):
  solver_path = '/media/6T/.tianle/.lib'
if os.path.exists(solver_path):
  sys.path.append(solver_path)
  from dl.utils.solver import Solver
  from dl.affinitynet.graph_attention import *
  from dl.affinitynet.test_graph_attention import *
  from dl.models.transformer import *

%load_ext autoreload
%autoreload 2

D:\ProgramFiles\Anaconda3\lib\site-packages\torch\nn\functional.py:52: UserWarning: size_average and reduce args will be deprecated, please use reduction='sum' instead.
  warnings.warn(warning.format(ret))


In [3]:
use_cuda = torch.cuda.is_available()
device = torch.device('cuda' if use_cuda else 'cpu')
if use_cuda:
  dtype = {'float': torch.cuda.FloatTensor, 'long': torch.cuda.LongTensor, 'byte': torch.cuda.ByteTensor} 
else:
  dtype = {'float': torch.FloatTensor, 'long': torch.LongTensor, 'byte': torch.ByteTensor} 

In [4]:
def log_rank_test(time, event, group):
  unique_time = time[event].unique()
  mask_time = (time >= unique_time.unsqueeze(-1))
  mask_time_point = (time == unique_time.unsqueeze(-1))

  num_groups = group.max().item()+1
  mask_group = Variable(torch.LongTensor(range(num_groups)).type(dtype['long'])).unsqueeze(-1) == group

  num_risk = mask_time.float().sum(1, keepdim=True)
  num_risk_group = torch.mm(mask_time.float(), mask_group.t().float())
  assert num_risk[-1]>1, ('The last time point only has only {1} object at rist,' 
                          'required at least 2!'.format(num_risk[-1].item()))
  
  num_event = torch.matmul(mask_time_point.float(), event.float()).unsqueeze(-1)
  num_event_group = torch.mm(mask_time_point.float(), (mask_group*event).t().float())
  
  expected_event_group = num_event * num_risk_group/num_risk

  o_e = (num_event_group.sum(0)-expected_event_group.sum(0))[1:]

  t1 = (num_event * (num_risk-num_event))/(num_risk**2 * (num_risk-1))
  t = num_risk.unsqueeze(-1) * torch.eye(num_groups-1)
  # based on intuition; not checked carefully; potential bug
  t2 = (t - num_risk_group[:, 1:].unsqueeze(-2))*num_risk_group[:, 1:].unsqueeze(-1) 
  var = (t1.unsqueeze(-1) * t2).sum(0)

  chi_squared = torch.matmul(torch.matmul(o_e, var.inverse()), o_e)

  return chi_squared
  

class CoxRegression(nn.Module):
  """
  """
  def __init__(self, in_dim, bias=False, time=None, event=None):
    super(CoxRegression, self).__init__()
    self.linear = nn.Linear(in_dim, 1, bias=bias)
    self.time = time
    self.event = event
  
  def forward(self, x, time=None, event=None):
    if time is None:
      time = self.time
    if event is None:
      event = self.event
    time, idx = time.sort()
    event = event[idx]
    x = x[idx]
    y = self.linear(x)
    n = y.size(0)
    t = y * torch.ones(n)
    t[torch.LongTensor(range(n)).unsqueeze(-1) < torch.LongTensor(range(n))] = float('-inf')
    m = t.max(dim=0)[0]
    nll = ((t-m).t()[event].exp().sum(1).log() + m[event]).sum() - y[event].sum()
#     nll = 0
#     for i in range(n):
#       if event[i]:
#         m = y[i:].max()
#         nll = nll + m + (y[i:] - m).exp().sum().log()
#     nll = nll - y[event].sum()
    return nll

In [5]:
def compare_classifiers(X_train, X_test, y_train, y_test, names, classifiers, res=None):
  def eval_classifiers(X_test, y_test):
    acc_test = []
    nmi_test = []
    f1_score_test = []
    confusion_mat_test = []
    for name, clf in zip(names, classifiers):
        clf.fit(X_train, y_train)
        y_pred = clf.predict(X_test)
        acc = sklearn.metrics.accuracy_score(y_true=y_test, y_pred=y_pred)
        nmi = sklearn.metrics.adjusted_mutual_info_score(labels_true=y_test, labels_pred=y_pred)
        confusion_mat = sklearn.metrics.confusion_matrix(y_test, y_pred)
        f1_score = sklearn.metrics.f1_score(y_true=y_test, y_pred=y_pred, average='weighted')
        print('{0}: acc={1:.2f}, nmi={2:.2f}, f1={3:.2f}, confusion_mat:\n{4}'.format(name, acc, nmi, 
                                                                        f1_score, confusion_mat))
        acc_test.append(acc)
        nmi_test.append(nmi)
        f1_score_test.append(f1_score)
        confusion_mat_test.append(confusion_mat)
    return acc_test, nmi_test, f1_score_test, confusion_mat_test
  
  acc_test, nmi_test, f1_score_test, confusion_mat_test = eval_classifiers(X_test, y_test)
  acc_train, nmi_train, f1_score_train, confusion_mat_train = eval_classifiers(X_train, y_train)
  if res is not None:
    acc_test = acc_test + res['test']['acc']
    nmi_test = nmi_test + res['test']['nmi']
    f1_score_test = f1_score_test + res['test']['f1_score']
    confusion_mat_test = confusion_mat_test + res['test']['confusion_mat']
    acc_train = acc_train + res['train']['acc']
    nmi_train = nmi_train + res['train']['nmi']
    f1_score_train = f1_score_train + res['train']['f1_score']
    confusion_mat_train = confusion_mat_train + res['train']['confusion_mat']
  res = {'train': {'acc': acc_train, 'nmi': nmi_train, 'f1_score': f1_score_train, 
                  'confusion_mat': confusion_mat_train},
        'test': {'acc': acc_test, 'nmi': nmi_test, 'f1_score': f1_score_test, 
                  'confusion_mat': confusion_mat_test}}
  return res

from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import RBF
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.naive_bayes import GaussianNB

names = ["Neural Net", "Decision Tree", "AdaBoost", "Nearest Neighbors", "Linear SVM", 
           "RBF SVM", "Random Forest", "Naive Bayes"]

classifiers = [
      MLPClassifier(alpha=1),
      DecisionTreeClassifier(max_depth=5),
      AdaBoostClassifier(),
      KNeighborsClassifier(3),
      SVC(kernel="linear"),
      SVC(gamma=2, C=1),
      RandomForestClassifier(max_depth=5, n_estimators=10, max_features=1),
      GaussianNB()]

In [6]:
def plot_feature_weight(feature_weight_all, colors, title, save_folder='.'):
  plt.figure(figsize=(figsize, figsize))
  plt.scatter(range(len(feature_weight_all)), feature_weight_all, c=colors, s=20)
  plt.xlabel('index')
  plt.ylabel('weight')
  if save_fig:
      if not os.path.exists(save_folder):
          os.makedirs(save_folder)
      plt.savefig('{0}/{1}.png'.format(save_folder, title), bbox_inches='tight', dpi=200)
  else:
      plt.title(title)
      plt.show()
  plt.close()

def plot_feature_weight_affinitynet(model_layers, title, save_folder='.'):
  print(title)
  feature_weight_all = np.ones(in_dim)
  for i in range(1):
      feature_weight = nn.functional.softmax(model_layers[i].components[0].a, 0).data.numpy()
      print('layer{0}: {1}'.format(i, feature_weight[:signal_dim].sum()/feature_weight.sum()))
      if signal_dim < 10:
        print(feature_weight[:signal_dim].tolist())
      feature_weight_all *= feature_weight
  feature_weight_all /= feature_weight_all.sum()
  print('overall: {0}'.format(feature_weight_all[:signal_dim].sum()))
  if signal_dim < 10:
    print(feature_weight_all[:signal_dim].tolist())
  colors = ['r']*signal_dim + ['b']*noisy_dim
  plot_feature_weight(feature_weight_all, colors, title, save_folder=save_folder)

def plot_feature_weight_linear(model, title, save_folder='.'):
  print(title)
  feature_weight = model.layers.linear0.weight
  for i in range(1, 1+len(model.layers)//2):
    feature_weight = torch.mm(getattr(model.layers, 'linear'+str(i)).weight, feature_weight)
  feature_weight = feature_weight.data.abs().mean(0).numpy()
  feature_weight_all = feature_weight / feature_weight.sum()
  print('overall: {0}'.format(feature_weight_all[:signal_dim].sum()))
  if signal_dim < 10:
    print(feature_weight_all[:signal_dim].tolist())
  colors = ['r']*signal_dim + ['b']*noisy_dim
  plot_feature_weight(feature_weight_all, colors, title, save_folder=save_folder)

def plot_result(loss_train, acc_train, loss_val, acc_val, avg='avg', 
                title_prefix='training-affinitynet', save_folder='.', save_fig=False):
  title = '{0}_best_val_acc_{1}={2}'.format(title_prefix, avg, acc_val[avg][-1])
  plt.figure(figsize=(figsize, figsize))
  plt.subplot(211)
  plt_loss_train, = plt.plot(loss_train[avg], 'r--')
  plt_loss_val, = plt.plot(loss_val[avg], 'g-')
  plt.legend([plt_loss_train, plt_loss_val], ['train', 'validation'], loc=0)
  plt.ylabel('loss')
  plt.subplot(212)
  plt_acc_train, = plt.plot(acc_train[avg], 'r--')
  plt_acc_val, = plt.plot(acc_val[avg], 'g-')
  plt.legend([plt_acc_train, plt_acc_val], ['train', 'validation'], loc=0)
  plt.ylabel('accuracy %')
  plt.xlabel('iterations')
  if save_fig:
      if not os.path.exists(save_folder):
          os.makedirs(save_folder)
      plt.savefig('{0}/{1}.png'.format(save_folder, title), bbox_inches='tight', dpi=200)
  else:
      plt.title(title)
      plt.show()
  plt.close()
  
def eval_affinitynet(data, res, train_portion=0.1, seed=0, k=2, batch_size=10, save_folder='.',  
                     save_model=True, model=None, verbose=False, silent=True, save_fig=False,
                    hidden_dims=[1000, 10], allow_duplicate=True):
  loss_fn = nn.CrossEntropyLoss()
  optimizer = torch.optim.Adam(model.parameters(), lr=lr, weight_decay=weight_decay)
  solver = Solver(model, data, optimizer, loss_fn)
  loss_train, acc_train, loss_val, acc_val = solver.train_eval(
    num_iter=num_iter, batch_size=batch_size,X=None, y=None, X_val=None, y_val=None,
    X_test=None, y_test=None, eval_test=False, balanced_sample=True, allow_duplicate=allow_duplicate)

  if not silent:
    if not os.path.exists(save_folder):
      os.makedirs(save_folder)
    loss_file = '{0}/loss-acc-affinitynet.pkl'.format(save_folder)
    with open(loss_file, 'wb') as f:
      pickle.dump({'loss_train': loss_train, 'acc_train': acc_train, 
                  'loss_val': loss_val, 'acc_val': acc_val}, f)
    plot_result(loss_train, acc_train, loss_val, acc_val, avg='avg', save_folder=save_folder, 
                save_fig=save_fig)
    plot_result(loss_train, acc_train, loss_val, acc_val, avg='batch', save_folder=save_folder, 
                save_fig=save_fig)

  acc, nmi, confusion_mat, f1_score = visualize_val(
    data['X_train'], data['y_train'], solver, batch_size=batch_size, 
    title='affinitynet X_train', topk=1, save_fig=save_fig, save_folder=save_folder, silent=silent)
  res[train_portion]['train']['acc'].append(acc)
  res[train_portion]['train']['nmi'].append(nmi)
  res[train_portion]['train']['f1_score'].append(f1_score)
  res[train_portion]['train']['confusion_mat'].append(confusion_mat)

  acc, nmi, confusion_mat, f1_score = visualize_val(
    data['X_val'], data['y_val'], solver, batch_size=batch_size, title='affinitynet X_val', topk=1,
    save_fig=save_fig, save_folder=save_folder, silent=silent)
  res[train_portion]['test']['acc'].append(acc)
  res[train_portion]['test']['nmi'].append(nmi)
  res[train_portion]['test']['f1_score'].append(f1_score)
  res[train_portion]['test']['confusion_mat'].append(confusion_mat)

  if save_model:
    if not os.path.exists(save_folder):
      os.makedirs(save_folder)
    torch.save(model.state_dict(), '{0}/model_state.pt'.format(save_folder))
    torch.save(model, '{0}/model.pt'.format(save_folder))
  if not silent:
    cnt = 0
    for n, p in model.named_parameters():
      if verbose:
        print(n, p.numel())
      cnt += p.numel()
    print('AffinityNet total params:{0}'.format(cnt))
  
  # hidden_dims = model[0].hidden_dims
  num_cls = data['y_train'].max().item()+1
  model = DenseLinear(hidden_dims[0], hidden_dims[1:]+[num_cls], dense=False).to(device)
  if not silent:
    cnt = 0
    for n, p in model.named_parameters():
      if verbose:
        print(n, p.numel())
      cnt += p.numel()
    print('DenseLinear: total params:{0}'.format(cnt))    

    
  loss_fn = nn.CrossEntropyLoss()
  optimizer = torch.optim.Adam(model.parameters(), lr=lr*0.5, weight_decay=weight_decay)
  solver = Solver(model, data, optimizer, loss_fn)
  loss_train, acc_train, loss_val, acc_val = solver.train_eval(
    num_iter=num_iter, batch_size=batch_size, X=None, y=None, X_val=None, y_val=None,
    X_test=None, y_test=None, eval_test=False, balanced_sample=True, allow_duplicate=allow_duplicate)

  if not silent:
    if not os.path.exists(save_folder):
      os.makedirs(save_folder)
    loss_file = '{0}/loss-acc-linear.pkl'.format(save_folder)
    with open(loss_file, 'wb') as f:
      pickle.dump({'loss_train': loss_train, 'acc_train': acc_train, 
                  'loss_val': loss_val, 'acc_val': acc_val}, f)
    plot_result(loss_train, acc_train, loss_val, acc_val, avg='avg', title_prefix = 'training-linear',
               save_folder=save_folder, save_fig=save_fig)
    plot_result(loss_train, acc_train, loss_val, acc_val, avg='batch', 
                title_prefix = 'training-linear', save_folder=save_folder, save_fig=save_fig)

  acc, nmi, confusion_mat, f1_score = visualize_val(
    data['X_train'], data['y_train'], solver, batch_size=batch_size, 
    title='linear X_train', topk=1, save_fig=save_fig, save_folder=save_folder, silent=silent)
  res[train_portion]['train']['acc'].append(acc)
  res[train_portion]['train']['nmi'].append(nmi)
  res[train_portion]['train']['f1_score'].append(f1_score)
  res[train_portion]['train']['confusion_mat'].append(confusion_mat)

  acc, nmi, confusion_mat, f1_score = visualize_val(
    data['X_val'], data['y_val'], solver, batch_size=batch_size, title='linear X_val', topk=1,
    save_fig=save_fig, save_folder=save_folder, silent=silent)
  res[train_portion]['test']['acc'].append(acc)
  res[train_portion]['test']['nmi'].append(nmi)
  res[train_portion]['test']['f1_score'].append(f1_score)
  res[train_portion]['test']['confusion_mat'].append(confusion_mat)

def run(x, y, train_portions=[0.01, 0.05, 0.1, 0.2, 0.4, 0.8], seed=0, batch_size=10, knn=3, 
        hidden_dims=None, ablation_name=None, silent=True, save_model=False, save_fig=False, 
        model_type='encoder', merge=None, merge_type='sum', layer_norm=True, 
        allow_duplicate=True):
  x_var = Variable(torch.Tensor(x).type(dtype['float']))
  y_var = Variable(torch.Tensor(y).type(dtype['long']))
  in_dim = x.shape[1]
  num_cls = len(np.unique(y))
  if hidden_dims is None:
    hidden_dims = [in_dim, 10]
  else:
    hidden_dims = [in_dim] + hidden_dims
  group_index = [range(x.shape[1])]
  
  res = {}
  for train_portion in train_portions:
    res[train_portion] = {}
    for i in ['train', 'test']:
      res[train_portion][i] = {}
      for j in ['acc', 'nmi', 'f1_score', 'confusion_mat']:
        res[train_portion][i][j] = []

  for train_portion in train_portions:
    proportions = [train_portion]*num_cls
    x_train, y_train, x_test, y_test, train_idx, test_idx = split_data(
        x_var, y_var, proportions=proportions, seed=seed)
    print('train size: {0}, test size: {1}'.format(y_train.size(0), y_test.size(0)))

    data = {'X_train':x_train.data, 'y_train':y_train.data, 'X_val':x_test.data, 'y_val':y_test.data, 
           'X_test':x_test.data, 'y_test':y_test.data}

    res[train_portion] = compare_classifiers(x_train.data.cpu().numpy(), x_test.data.cpu().numpy(), 
                                            y_train.data.cpu().numpy(), y_test.data.cpu().numpy(), 
                                             names, classifiers)
#     model = MultiviewAttention(in_dim=in_dim, hidden_dims=hidden_dims, k=k, graph=None, 
#                                out_indices=None, feature_subset=None, kernel='affine', 
#                                nonlinearity_1=nn.ReLU(), nonlinearity_2=None, 
#                                use_previous_graph=True,
#                                group_index=group_index, 
#                                merge=None, merge_type='affine', 
#                                reset_graph_every_forward=True, 
#                                no_feature_transformation=[True] + [False]*(len(hidden_dims)-1), 
#                                rescale=True, merge_dim=hidden_dims[-1], 
#                                layer_norm=True)
#     model = nn.Sequential(model, nn.Linear(hidden_dims[-1], num_cls))
    if model_type == 'encoder':
      out_dim = 10 
      key_dim = 10
      value_dim = 10 
      num_heads = 1
      mask=False 
      query_in_dim=None

      fc_dim = 10
      linear_dim = 10
      num_attention = 2
      return_graph = False
      return_all = False

      model = nn.Sequential(WeightedFeature(in_dim), 
                            MultiheadAttention(in_dim, out_dim, key_dim, value_dim, num_heads, knn=knn),
                            StackedEncoder(out_dim, key_dim, value_dim, fc_dim, linear_dim, num_cls, num_heads, 
                             num_attention, knn, return_graph=return_graph, 
                                           return_all=return_all)).to(device)  
    else: # not good
#       hidden_dims = [in_dim , 10]
#       group_index = [range(in_dim)]
      model = MultiviewAttention(in_dim=in_dim, hidden_dims=hidden_dims, k=knn, graph=None, 
                                 out_indices=None, feature_subset=None, kernel='cosine', 
                                 nonlinearity_1=nn.ReLU(), nonlinearity_2=None, 
                                 use_previous_graph=True,
                                 group_index=group_index, 
                                 merge=merge, merge_type=merge_type, 
                                 reset_graph_every_forward=True, 
                                 no_feature_transformation=[True] + [False]*(len(hidden_dims)-1), 
                                 rescale=True, merge_dim=hidden_dims[-1], 
                                 layer_norm=layer_norm)
      model = nn.Sequential(model, nn.Linear(hidden_dims[-1], num_cls)).to(device)     
#       print(in_dim, hidden_dims, group_index, merge, merge_type, layer_norm)
#       for n, p in model[0].layers[0].named_parameters():
#         print(n, p)
  
    if not silent:
      if x_var.numel() < 10**4:
          title = 'raw data PCA'
          plot_scatter(x_var, title=title, colors=y, folder=save_folder, save_fig=save_fig, 
                      size=figsize)
          y_pred = model(x_var)
          title = 'before training output PCA'
          plot_scatter(y_pred, title=title, colors=y, folder=save_folder, save_fig=save_fig, 
                      size=figsize)
    
    if ablation_name is None:
      ablation_name = 'seed'
    save_folder = '{0}/{1}-{2}/train_portion-{3}'.format(save_folder_prefix, ablation_name, 
                                                         eval(ablation_name), train_portion)
    
    eval_affinitynet(data, res, train_portion=train_portion, seed=seed, batch_size=batch_size, 
                     save_folder=save_folder, model=model, save_model=save_model, silent=silent,
                    save_fig=save_fig, hidden_dims=hidden_dims, allow_duplicate=allow_duplicate)

  res_file = '{0}/{1}-{2}'.format(save_folder_prefix, ablation_name, eval(ablation_name))
  if not os.path.exists(res_file):
    os.makedirs(res_file)
  print(res_file)
  res_file = '{0}/res.pkl'.format(res_file)
  with open(res_file, 'wb') as f:
    pickle.dump(res, f)

hidden_dims = [in_dim] + HIDDEN_DIMS
group_index = [range(x.shape[1])]
batch_size = 10
allow_duplicate = True
train_portion = 0.1
lr = 1e-2
proportions = [train_portion]*num_cls
x_train, y_train, x_test, y_test, train_idx, test_idx = split_data(
    x_var, y_var, proportions=proportions, seed=seed)
print('train size: {0}, test size: {1}'.format(y_train.size(0), y_test.size(0)))

data = {'X_train':x_train.data, 'y_train':y_train.data, 'X_val':x_test.data, 'y_val':y_test.data, 
       'X_test':x_test.data, 'y_test':y_test.data}

model = MultiviewAttention(in_dim=in_dim, hidden_dims=hidden_dims, k=knn, graph=None, 
                               out_indices=None, feature_subset=None, kernel='cosine', 
                               nonlinearity_1=nn.ReLU(), nonlinearity_2=None, 
                               use_previous_graph=True,
                               group_index=group_index, 
                               merge=merge, merge_type=merge_type, 
                               reset_graph_every_forward=True, 
                               no_feature_transformation=[True] + [False]*(len(hidden_dims)-1), 
                               rescale=True, merge_dim=hidden_dims[-1], 
                               layer_norm=layer_norm)
model = nn.Sequential(model, nn.Linear(hidden_dims[-1], num_cls)).to(device)

loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=lr, weight_decay=weight_decay)
solver = Solver(model, data, optimizer, loss_fn)

loss_train, acc_train, loss_val, acc_val = solver.train_eval(
  num_iter=num_iter, batch_size=batch_size,X=None, y=None, X_val=None, y_val=None,
  X_test=None, y_test=None, eval_test=False, balanced_sample=True, allow_duplicate=False)

acc, nmi, confusion_mat, f1_score = visualize_val(
    data['X_train'], data['y_train'], solver, batch_size=batch_size, 
    title='affinitynet X_train', topk=1, save_fig=False, save_folder='.', silent=False)

y = solver.predict(batch_size=batch_size, save_file=False, 
                    X=x_train, y=y_train, topk=1)

from dl.utils.sampler import BatchLoader
dataloader_train = BatchLoader((x_train, y_train), batch_size, balanced=True, 
    num_iter=2, allow_duplicate=False, max_redundancy=1000, 
    shuffle=True, seed=None)
list(dataloader_train)

num_iters = 200
loss_train = []
loss_val = []
acc_train = []
acc_val = []
verbose = True

lr = 1e-2
weight_decay = 1e-4
optimizer = torch.optim.Adam(model.parameters(), lr=lr, weight_decay=weight_decay)
loss_fn = nn.CrossEntropyLoss()

for i in range(num_iters):
  y_pred = model(x_train)
  loss = loss_fn(y_pred, y_train)
  loss_train.append(loss.item())
  acc_train.append(eval_acc(None, None, y_train, y_pred[0] if isinstance(y_pred, tuple) else y_pred, 
                            return_value=True, verbose=False)[0])
  optimizer.zero_grad()
  loss.backward()
  optimizer.step()
  y_pred = model(x_test)
  loss = loss_fn(y_pred, y_test)
  loss_val.append(loss.item())
  acc_val.append(eval_acc(None, None, y_test, y_pred[0] if isinstance(y_pred, tuple) else y_pred, 
                          return_value=True, verbose=False)[0])

plot_loss_acc(loss_train, acc_train, loss_val, acc_val,
            title_prefix='training-affinitynet', save_folder='.', save_fig=False)

In [7]:
#   model = MultiviewAttention(in_dim=in_dim, hidden_dims=hidden_dims, k=knn, graph=None, 
#                                  out_indices=None, feature_subset=None, kernel='cosine', 
#                                  nonlinearity_1=nn.ReLU(), nonlinearity_2=None, 
#                                  use_previous_graph=True,
#                                  group_index=group_index, 
#                                  merge=merge, merge_type=merge_type, 
#                                  reset_graph_every_forward=True, 
#                                  no_feature_transformation=[True] + [False]*(len(hidden_dims)-1), 
#                                  rescale=True, merge_dim=hidden_dims[-1], 
#                                  layer_norm=layer_norm)
#   model = nn.Sequential(model, nn.Linear(hidden_dims[-1], num_cls)).to(device)

# for n, p in model[0].layers[0].named_parameters():
#   print(n, p)

# Prepare data

In [8]:
root = '../'
num_iter = 200
# knn = 3
knn = 2
# hidden_dims = [50, 10]
# hidden_dims = [50, 50]
hidden_dims = [100]
cancer_type = 'kidney'
oncogrid = ''
data_type = 0
# oncogrid = 'oncogrid'
# data_type = 3
feature_cat = False
# seeds = range(30, 40)
seeds = range(40, 50)
model_type = 'affinitynet-h{0}-k{1}-i{2}'.format('-'.join([str(i) for i in hidden_dims]), 
                                                          knn, num_iter)
if oncogrid != '':
  model_type = '{0}-{1}'.format(model_type, oncogrid)

# assume it is good to make gene expression sum to 100 for each sample (does not check if it is good)
if data_type==3 and (not feature_cat):
  layer_norm = [[True, False]] * len(hidden_dims) + [[False, False]]
elif data_type==3:
  layer_norm = False
else:
  layer_norm = True

merge=None
merge_type='sum'
allow_duplicate = True
batch_size = 10
normalization = 1
noisy_dim = 0

lr = 1e-2
weight_decay = 1e-4

figsize = 10
save_fig = True
save_model = True

train_portions=[0.01, 0.05, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8]

# data = pickle.load(open('{0}/data/gene_mirnas_5cancer.pkl'.format(root), 'rb'))
# gene, mirnas, y = data[cancer_type]
if oncogrid == '':
  data = pickle.load(open('{0}/data/gene_mirnas_5cancer-new.pkl'.format(root), 'rb'))
  gene, mirnas, y, sample_ids, gene_ids, mirnas_ids = data[cancer_type]
else:
  file = '{0}/data/{1}-oncogrid.npz'.format(root, cancer_type)
  print(file)
  npzfile = np.load(file)
  gene=npzfile['gene'] 
  mirnas=npzfile['mirnas']
  sample_ids=npzfile['sample_ids']
  gene_ids=npzfile['gene_ids'] 
  sample_gene_mut=npzfile['sample_gene_mut']
  mirnas_ids=npzfile['mirnas_ids']
  print('gene:{0}, mirnas:{1}, sample_ids:{2}, gene_ids:{3}, sample_gene_mut:{4}, mirnas_ids:{5}'.format(
      gene.shape, mirnas.shape, sample_ids.shape, gene_ids.shape, sample_gene_mut.shape, mirnas_ids.shape))
  y = sample_ids[:,1]
  print(collections.Counter(sample_ids[:,1]))
  label_code = {v:k for k, v in enumerate(np.unique(y))}
  y = np.array([label_code[i] for i in y], dtype=np.int)

if normalization == 1:
  gene = gene / np.sum(gene, axis=1, keepdims=True) * 100
  mirnas = mirnas / np.sum(mirnas, axis=1, keepdims=True) * 100
if data_type == 0:
  x = gene
  group_index = [range(gene.shape[1])]
  data_type_name = 'gene'
elif data_type == 1:
  x = mirnas
  group_index = [range(mirnas.shape[1])]
  data_type_name = 'mirnas'
elif data_type == 2:
  x = np.concatenate([gene, mirnas], axis=1)
  if feature_cat:
    group_index = [range(gene.shape[1]+mirnas.shape[1])]
    data_type_name = 'gene-mirnas-cat'
  else:
    group_index = [range(gene.shape[1]), range(mirnas.shape[1])]
    data_type_name = 'gene-mirnas'
elif data_type == 3:
  x = np.concatenate([gene, sample_gene_mut], axis=1)
  if feature_cat:
    group_index = [range(gene.shape[1]+sample_gene_mut.shape[1])]
    data_type_name = 'gene-mut-cat'
  else:
    group_index = [range(gene.shape[1]), range(sample_gene_mut.shape[1])]
    data_type_name = 'gene-mut'
  
save_folder_prefix = '{0}/data/cancer/{4}/{3}/{1}/{2}/'.format(root, cancer_type, data_type_name, 
                                                           model_type, oncogrid)
if not os.path.exists(save_folder_prefix):
  os.makedirs(save_folder_prefix)

signal_dim = x.shape[1]
if noisy_dim > 0:
  z = np.random.multivariate_normal([np.mean(x)]*noisy_dim, np.std(x)*np.eye(noisy_dim), 
                                    size=len(y))
  x = np.concatenate([x, y],axis=1)
in_dim = x.shape[1]
num_cls = int(y.max())+1

# hidden_dims = [in_dim, 10]
# if data_type_name == 'gene-mirnas':
#   hidden_dims = [in_dim//2, 10]

x_var = Variable(torch.Tensor(x).type(dtype['float']))
y_var = Variable(torch.Tensor(y).type(dtype['long']))

In [9]:
start_time = time.time()
for seed in seeds:
  run(x, y, train_portions=train_portions, seed=seed, batch_size=batch_size, knn=knn, 
      hidden_dims=hidden_dims, ablation_name=None, silent=False, save_model=save_model, 
      save_fig=save_fig, 
      model_type=model_type, merge=merge, merge_type=merge_type, layer_norm=layer_norm,
     allow_duplicate=allow_duplicate)
end_time = time.time()
print('Time spent: {0}s'.format(end_time - start_time))

train size: 7, test size: 647


/projects/academic/azhang/projectmodules/anaconda3/lib/python3.6/site-packages/sklearn/neural_network/multilayer_perceptron.py:564: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


Neural Net: acc=0.94, nmi=0.73, f1=0.94, confusion_mat:
[[ 59   4   1]
 [  3 297  13]
 [  2  16 252]]
Decision Tree: acc=0.63, nmi=0.15, f1=0.62, confusion_mat:
[[ 12   0  52]
 [ 28 176 109]
 [  8  44 218]]
AdaBoost: acc=0.93, nmi=0.71, f1=0.93, confusion_mat:
[[ 56   3   5]
 [  3 290  20]
 [  2  10 258]]
Nearest Neighbors: acc=0.87, nmi=0.53, f1=0.85, confusion_mat:
[[ 14  49   1]
 [  3 304   6]
 [  2  26 242]]
Linear SVM: acc=0.93, nmi=0.72, f1=0.93, confusion_mat:
[[ 62   2   0]
 [  4 298  11]
 [  3  23 244]]
RBF SVM: acc=0.82, nmi=0.46, f1=0.78, confusion_mat:
[[  0   0  64]
 [  0 265  48]
 [  0   5 265]]
Random Forest: acc=0.81, nmi=0.37, f1=0.77, confusion_mat:
[[  0  59   5]
 [  0 286  27]
 [  0  34 236]]
Naive Bayes: acc=0.81, nmi=0.39, f1=0.77, confusion_mat:
[[  0  63   1]
 [  0 301  12]
 [  0  48 222]]


/projects/academic/azhang/projectmodules/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/projects/academic/azhang/projectmodules/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/projects/academic/azhang/projectmodules/anaconda3/lib/python3.6/site-packages/sklearn/neural_network/multilayer_perceptron.py:564: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


Neural Net: acc=1.00, nmi=1.00, f1=1.00, confusion_mat:
[[1 0 0]
 [0 3 0]
 [0 0 3]]
Decision Tree: acc=1.00, nmi=1.00, f1=1.00, confusion_mat:
[[1 0 0]
 [0 3 0]
 [0 0 3]]
AdaBoost: acc=1.00, nmi=1.00, f1=1.00, confusion_mat:
[[1 0 0]
 [0 3 0]
 [0 0 3]]


...

Random Forest: acc=0.88, nmi=0.57, f1=0.88, confusion_mat:
[[ 39   3   3]
 [  3 201  17]
 [  3  24 164]]
Naive Bayes: acc=0.91, nmi=0.63, f1=0.90, confusion_mat:
[[ 23   2  20]
 [  0 208  13]
 [  0   7 184]]


/projects/academic/azhang/projectmodules/anaconda3/lib/python3.6/site-packages/sklearn/neural_network/multilayer_perceptron.py:564: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


Neural Net: acc=0.99, nmi=0.97, f1=0.99, confusion_mat:
[[20  0  0]
 [ 1 94  0]
 [ 0  0 82]]
Decision Tree: acc=1.00, nmi=1.00, f1=1.00, confusion_mat:
[[20  0  0]
 [ 0 95  0]
 [ 0  0 82]]
AdaBoost: acc=1.00, nmi=1.00, f1=1.00, confusion_mat:
[[20  0  0]
 [ 0 95  0]
 [ 0  0 82]]
Nearest Neighbors: acc=0.97, nmi=0.86, f1=0.97, confusion_mat:
[[20  0  0]
 [ 2 92  1]
 [ 1  1 80]]
Linear SVM: acc=0.99, nmi=0.94, f1=0.99, confusion_mat:
[[20  0  0]
 [ 1 94  0]
 [ 0  1 81]]
RBF SVM: acc=1.00, nmi=1.00, f1=1.00, confusion_mat:
[[20  0  0]
 [ 0 95  0]
 [ 0  0 82]]
Random Forest: acc=0.99, nmi=0.94, f1=0.99, confusion_mat:
[[20  0  0]
 [ 1 94  0]
 [ 0  1 81]]
Naive Bayes: acc=0.97, nmi=0.86, f1=0.97, confusion_mat:
[[20  0  0]
 [ 1 92  2]
 [ 1  1 80]]
AffinityNet total params:101403
DenseLinear: total params:100403
train size: 261, test size: 393
Neural Net: acc=0.94, nmi=0.75, f1=0.94, confusion_mat:
[[ 37   0   2]
 [  3 179   8]
 [  0   9 155]]
Decision Tree: acc=0.90, nmi=0.61, f1=0.90, conf

/projects/academic/azhang/projectmodules/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


RBF SVM: acc=0.83, nmi=0.47, f1=0.79, confusion_mat:
[[  0   0  39]
 [  0 165  25]
 [  0   4 160]]
Random Forest: acc=0.92, nmi=0.67, f1=0.92, confusion_mat:
[[ 32   3   4]
 [  2 179   9]
 [  1  11 152]]
Naive Bayes: acc=0.90, nmi=0.63, f1=0.90, confusion_mat:
[[ 21   0  18]
 [  0 176  14]
 [  1   6 157]]
Neural Net: acc=0.99, nmi=0.93, f1=0.99, confusion_mat:
[[ 26   0   0]
 [  1 124   1]
 [  0   1 108]]
Decision Tree: acc=1.00, nmi=1.00, f1=1.00, confusion_mat:
[[ 26   0   0]
 [  0 126   0]
 [  0   0 109]]
AdaBoost: acc=0.80, nmi=0.54, f1=0.79, confusion_mat:
[[ 26   0   0]
 [  0 124   2]
 [  0  50  59]]
Nearest Neighbors: acc=0.97, nmi=0.84, f1=0.97, confusion_mat:
[[ 25   1   0]
 [  2 123   1]
 [  3   1 105]]
Linear SVM: acc=0.99, nmi=0.95, f1=0.99, confusion_mat:
[[ 26   0   0]
 [  1 125   0]
 [  0   1 108]]
RBF SVM: acc=1.00, nmi=1.00, f1=1.00, confusion_mat:
[[ 26   0   0]
 [  0 126   0]
 [  0   0 109]]
Random Forest: acc=0.99, nmi=0.95, f1=0.99, confusion_mat:
[[ 26   0   0]
 [

In [10]:
# data_folder = 'F:/TCGA/analysis/data'
# clinical_file = '{0}/clinical.cases_selection.2018-04-18/clinical.tsv'.format(data_folder)
# clinical = np.genfromtxt(clinical_file, delimiter='\t', dtype='S')
# surv_plot = np.genfromtxt('{0}/survival-plot.tsv'.format(data_folder), dtype='S')

# project_id = 'TCGA-KIRC'
# sample_type = '01'
# var_name = 'tumor_stage'
# data_types = np.array(['mirnas', 'ht_counts', 'methy'])

# x_cat = []
# y_cat = []
# case_ids = []

# for data_type in data_types[[1]]:
#   print(data_type)
#   input_file = '{0}/{1}/{2}_{3}-{4}'.format(data_folder, project_id, data_type, sample_type, var_name)
#   with np.load(input_file+'.npz') as npz:
#     x = npz['x']
#     y = npz['y']
#     labels = npz['labels']
#     sample_ids = npz['sample_ids']
#   x_cat.append(x)
#   y_cat.append(y)
#   case_ids.append(sample_ids)
#   print(collections.Counter(labels), np.bincount(y))

# save_folder_prefix = '{0}/{1}/results/{2}'.format(data_folder, project_id, data_type)
# if not os.path.exists(save_folder_prefix):
#   os.makedirs(save_folder_prefix)

# sample_ids = sorted(functools.reduce(lambda x, y: set(x).intersection(y), case_ids))

# x = []
# y = []
# for i in range(len(case_ids)):
#   idx = [list(case_ids[i]).index(s) for s in sample_ids]
#   x.append(x_cat[i][idx])
# y = y_cat[i][idx]
# x = np.concatenate(x, axis=1)

# idx_surv = np.array([[i, list(surv_plot[:, -1]).index(s)] for i, s in enumerate(sample_ids) 
#             if s in surv_plot[:, -1]])

# x = x[idx_surv[:,0]]
# y = y[idx_surv[:,0]]

def graph_loss(y_pred, y_var, a0=1, a1=1, verbose=True):
  loss0 = nn.CrossEntropyLoss()(y_pred[0], y_var)
  loss1 = (y_pred[1][y_var.unsqueeze(-1) != y_var].sum()) / y_var.numel()
  loss = a0*loss0 + a1*loss1
  if verbose:
    print('loss0 = {0:.2f}, loss1 = {1:.2f}'.format(loss0.item(), loss1.item()))
  return loss

def plot_loss_acc(loss_train, acc_train, loss_val, acc_val, 
                title_prefix='', save_folder='.', save_fig=False, figsize=10):
  title = '{0}_best_val_acc={1}'.format(title_prefix, np.max(acc_val))
  plt.figure(figsize=(figsize, figsize))
  plt.subplot(211)
  plt_loss_train, = plt.plot(loss_train, 'r--')
  plt_loss_val, = plt.plot(loss_val, 'g-')
  plt.legend([plt_loss_train, plt_loss_val], ['train', 'validation'], loc=0)
  plt.ylabel('loss')
  plt.subplot(212)
  plt_acc_train, = plt.plot(acc_train, 'r--')
  plt_acc_val, = plt.plot(acc_val, 'g-')
  plt.legend([plt_acc_train, plt_acc_val], ['train', 'validation'], loc=0)
  plt.ylabel('accuracy %')
  plt.xlabel('iterations')
  if save_fig:
      if not os.path.exists(save_folder):
          os.makedirs(save_folder)
      plt.savefig('{0}/{1}.png'.format(save_folder, title), bbox_inches='tight', dpi=200)
  else:
      plt.title(title)
      plt.show()
  plt.close()

out_dim = 10 
key_dim = 10
value_dim = 10 
num_heads = 2
mask=False 
query_in_dim=None
knn = 5

fc_dim = 10
linear_dim = 10
num_attention = 1
return_graph = False
return_all = False

model = nn.Sequential(WeightedFeature(in_dim), 
                  MultiheadAttention(in_dim, out_dim, key_dim, value_dim, num_heads, knn=knn),
                  StackedEncoder(out_dim, key_dim, value_dim, fc_dim, linear_dim, num_cls, num_heads, 
                   num_attention, knn, return_graph=return_graph, return_all=return_all)).to(device)  

train_portion = 0.5
seed = 0
proportions = [train_portion]*num_cls
x_train, y_train, x_test, y_test, train_idx, test_idx = split_data(
    x_var, y_var, proportions=proportions, seed=seed)
print('train size: {0}, test size: {1}'.format(y_train.size(0), y_test.size(0)))

num_iters = 10
loss_train = []
loss_val = []
acc_train = []
acc_val = []
verbose = True

lr = 1e-3
weight_decay = 1e-4
optimizer = torch.optim.Adam(model.parameters(), lr=lr, weight_decay=weight_decay)
loss_fn = nn.CrossEntropyLoss()

for i in range(num_iters):
  y_pred = model(x_train)
  loss = loss_fn(y_pred, y_train)
  loss_train.append(loss.item())
  acc_train.append(eval_acc(None, None, y_train, y_pred[0] if isinstance(y_pred, tuple) else y_pred, 
                            return_value=True, verbose=False)[0])
  optimizer.zero_grad()
  loss.backward()
  optimizer.step()
  y_pred = model(x_test)
  loss = loss_fn(y_pred, y_test)
  loss_val.append(loss.item())
  acc_val.append(eval_acc(None, None, y_test, y_pred[0] if isinstance(y_pred, tuple) else y_pred, 
                          return_value=True, verbose=False)[0])

plot_loss_acc(loss_train, acc_train, loss_val, acc_val,
            title_prefix='training-affinitynet', save_folder='.', save_fig=False)

y_pred = model(x_test)
eval_acc(None, None, y_test, y_pred[0] if isinstance(y_pred, tuple) else y_pred, 
         return_value=True, verbose=False)

y_pred = model(x_train)
eval_acc(None, None, y_train, y_pred[0] if isinstance(y_pred, tuple) else y_pred, 
         return_value=True, verbose=False)

plt.hist(model[0].weight.data.cpu().numpy())